# CAPSTONE PROJCECT—Text Analysis

---

In [1]:
# Import package
#!pip install beautifulsoup4
#!pip install google
#!pip install nbinteract
import os
import re 
import requests
import numpy as np
from termcolor import colored
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
from bs4 import *
from googlesearch import search

In [2]:
import anvil.server

anvil.server.connect("server_7QGV67FAA6TXHGRR5QIXGBMI-KX2TWHW6MPGCG7FD")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Published" as SERVER


---
## Open-Domain QA Model

In [3]:
@anvil.server.callable
# Define Open-domain QA model function
def get_answer(question):
    
#-------------------------------------------------------------------------------------
    # To search on google
    urls = [] 
    for j in search(question, tld="co.in", num=10, stop=10, pause=2):
        urls.append(j)
        
#-------------------------------------------------------------------------------------        
    # Get conext of each url

    contexts = []
    for i in range(len(urls)):
        url = urls[i]

        # Fetch URL Content
        r = requests.get(url)
        
        # Get body content
        soup = BeautifulSoup(r.text,'html.parser').select('body')[0]

        # Initialize variable
        paragraphs = []

        # Iterate through all tags
        for tag in soup.find_all():
            # Check each tag name
            # For Paragraph use p tag
            if tag.name=="p":

                # use text for fetch the content inside p tag
                paragraphs.append(tag.text)

        context = ''
        for i in range(len(paragraphs)):
            context = context + paragraphs[i]

        contexts.append(context)
            
            
#-------------------------------------------------------------------------------------        
        # Get url with max scores answer
    scores = []
    for c in range(len(urls)):
        context = contexts[c]
        model_name = "thatdramebaazguy/roberta-base-squad"
        nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
        QA_input = {
            'question': question,
            'context': context
        }
        res = nlp(QA_input)
        score = res.get("score")
        scores.append(score)
    max_scores = max(scores)
    max_index = scores.index(max_scores)
    target_url = urls[max_index]        

#-------------------------------------------------------------------------------------        
        # Get predictions
    context = contexts[max_index]
    model_name = "thatdramebaazguy/roberta-base-squad"
    nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
    QA_input = {
        'question': question,
        'context': context
    }
    res = nlp(QA_input)

    # Locate answer 
    s_start = res.get('start')-300
    if s_start < 0:
        s_start = 0
    #sentence = '...'+ context[s_start:res.get('start')] + colored(res.get("answer"), 'green', attrs=['reverse', 'blink']) + context[res.get('end'):res.get('end')+300]+'...'
    forward_context = '...'+ context[s_start:res.get('start')]
    back_context = context[res.get('end'):res.get('end')+300]+'...'
    answer = res.get("answer")
    score = res.get("score")
    #print(f'CONTEXT: {sentence} \nQESTION: {question} \nANSWER: {res.get("answer")} \nSCORE: {res.get("score")} \nURL: {target_url}')
    print(f'ANSWER: {res.get("answer")}')
    return forward_context, back_context, answer, score, target_url




---
## Closed-Domain QA Model

In [4]:
@anvil.server.callable
# Define Closed-domain QA model function
def get_answer2(context, question):
    model_name = "thatdramebaazguy/roberta-base-squad"
    nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
    QA_input = {
        'question': question,
        'context': context
    }
    res = nlp(QA_input)
    
    s_start = res.get('start')-300
    if s_start < 0:
        s_start = 0
    forward_context = '...'+ context[s_start:res.get('start')]
    back_context = context[res.get('end'):res.get('end')+300]+'...'
    answer = res.get("answer")
    score = res.get("score")
    print(f'ANSWER: {res.get("answer")}')
    return forward_context, back_context, answer, score

/Users/chenlei/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


ANSWER: Heat
ANSWER: Heat
ANSWER: Heat
ANSWER: Miami
